Persistencia y particionado
----------------------

### Persistencia (*caching*)

Problema al usar un RDD varias veces:

-   Spark recomputa el RDD y sus dependencias cada vez que se ejecuta
    una acción

-   Muy costoso (especialmente en problemas iterativos)

Solución

-   Conservar el RDD en memoria y/o disco

-   Métodos `cache()` o `persist()`

#### Niveles de persistencia
| Nivel                | Espacio  | CPU     | Memoria/Disco   | Descripción
| :------------------: | :------: | :-----: | :-------------: | ------------------
| MEMORY_ONLY          |   Alto   |   Bajo  |     Memoria     | Guarda el RDD como un objeto Java no serializado en la JVM. Si el RDD no cabe en memoria, algunas particiones no se *cachearán* y serán recomputadas "al vuelo" cada vez que se necesiten. Nivel por defecto en Java y Scala.
| MEMORY_ONLY_SER      |   Bajo   |   Alto  |     Memoria     | Guarda el RDD como un objeto Java serializado (un *byte array* por partición). Nivel por defecto en Python, usando [`pickle`](http://docs.python.org/2/library/pickle.html).
| MEMORY_AND_DISK      |   Alto   |   Medio |     Ambos       | Guarda el RDD como un objeto Java no serializado en la JVM. Si el RDD no cabe en memoria, las particiones que no quepan se guardan en disco y se leen del mismo cada vez que se necesiten
| MEMORY_AND_DISK_SER  |   Bajo   |   Alto  |     Ambos       | Similar a MEMORY_AND_DISK pero usando objetos serializados.
| DISK_ONLY            |   Bajo   |   Alto  |     Disco       | Guarda las particiones del RDD solo en disco.
| OFF_HEAP             |          |         |                 | Guarda el RDD serializado usando [`Tachyon`](http://tachyon-project.org/).
   
Definidos en
[`pyspark.StorageLevel`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.StorageLevel)
    
#### Nivel de persistencia

-   En Scala y Java, el nivel por defecto es MEMORY\_ONLY

-   En Python, el nivel por defecto es MEMORY\_ONLY\_SER

    -   Por defecto, Python serializa los datos como objetos *pickled*

- Es posible especificar otra serialización al crear el SparkContext
```python
sc = SparkContext(master="local", appName="Mi app", serializer=pyspark.MarshalSerializer())
```
    
#### Recuperación de fallos

-   Si falla un nodo con datos almacenados, el RDD se recomputa

    -   Añadiendo `_2` al nivel de persistencia, se guardan 2 copias del
        RDD
        
#### Gestión de la cache

-   Algoritmo LRU para gestionar la cache

    -   Para niveles *solo memoria*, los RDDs viejos se eliminan y
        recalculan

    -   Para niveles *memoria y disco*, las particiones que no caben se
        escriben a disco

In [1]:
from test_helper import Test
from __future__ import print_function

rdd = sc.parallelize(range(1000), 10)
Test.assertTrue(not rdd.is_cached)

rdd.persist(StorageLevel.MEMORY_AND_DISK_SER_2)
Test.assertTrue(rdd.is_cached)

rdd2 = rdd.map(lambda x: x*x)
Test.assertTrue(not rdd2.is_cached)

rdd2.cache() # Nivel por defecto
Test.assertTrue(rdd2.is_cached)     

print("Nivel de persistencia de rdd: {0} ".format(rdd.getStorageLevel()))
print("Nivel de persistencia de rdd2: {0}".format(rdd2.getStorageLevel()))

rdd.unpersist() # Sacamos rdd de la cache
Test.assertTrue(not rdd.is_cached)

1 test passed.
1 test passed.
1 test passed.
1 test passed.
Nivel de persistencia de rdd: Disk Memory Serialized 2x Replicated 
Nivel de persistencia de rdd2: Memory Serialized 1x Replicated
1 test passed.


### Particionado

El número de particiones es función del tamaño del cluster o el número de
bloques del fichero en HDFS

-   Es posible ajustarlo al crear u operar sobre un RDD

-   El paralelismo de RDDs que derivan de otros depende del de sus RDDs
    padre

-   Dos funciones útiles:

    -   `rdd.getNumPartitions()` devuelve el número de particiones del
        RDD

    -   `rdd.glom()` devuelve un nuevo RDD juntando los elementos de
        cada partición en una lista


In [2]:
rdd = sc.parallelize([1, 2, 3, 4, 2, 4, 1], 4)
pairs = rdd.map(lambda x: (x, x))
Test.assertEquals(pairs.collect(), [(1,1),(2,2),(3,3),(4,4),(2,2),(4,4),(1,1)])
Test.assertEquals(pairs.getNumPartitions(), 4)

print("Pairs con 4 particiones: {0}".format(
        pairs.glom().collect()))

# Reducción manteniendo el número de particiones
print("Reducción con 4 particiones: {0}".format(
        pairs.reduceByKey(lambda x, y: x+y).glom().collect()))
# Reducción modificando el número de particiones

print("Reducción con 2 particiones: {0}".format(
        pairs.reduceByKey(lambda x, y: x+y, 2).glom().collect()))

1 test passed.
1 test passed.
Pairs con 4 particiones: [[(1, 1)], [(2, 2), (3, 3)], [(4, 4), (2, 2)], [(4, 4), (1, 1)]]
Reducción con 4 particiones: [[(4, 8)], [(1, 2)], [(2, 4)], [(3, 3)]]
Reducción con 2 particiones: [[(2, 4), (4, 8)], [(1, 2), (3, 3)]]


#### Funciones de reparticionado
- `repartition(n)` devuelve un nuevo RDD que tiene exactamente `n` particiones
- `coalesce(n)` más eficiente que `repartition`, minimiza el movimiento de datos
    - Solo permite reducir el número de particiones
- `partitionBy(n,[partitionFunc])` Particiona por clave, usando una función de particionado (por defecto, un hash de la clave)
    - Para RDDs clave/valor
    - La misma clave a la misma partición

In [3]:
print("Pairs con 5 particiones: {0}".format(
        pairs.repartition(5).glom().collect()))

print("Pairs con 2 particiones: {0}".format(
        pairs.coalesce(2).glom().collect()))

print("Particionado por clave (2 particiones): {0}".format(
        pairs.partitionBy(2).glom().collect()))

Pairs con 5 particiones: [[(2, 2), (4, 4)], [(1, 1), (2, 2), (1, 1)], [(3, 3)], [], [(4, 4)]]
Pairs con 2 particiones: [[(1, 1), (2, 2), (3, 3)], [(4, 4), (2, 2), (4, 4), (1, 1)]]
Particionado por clave (2 particiones): [[(2, 2), (4, 4), (2, 2), (4, 4)], [(1, 1), (3, 3), (1, 1)]]
